In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=0)

In [9]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
   
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    concat = np.zeros((n_a+n_x, m))
    concat[:n_a, :] = a_prev
    concat[n_a:, :] = xt
    
    ft = sigmoid(np.dot(Wf, concat) + bf)
    it = sigmoid(np.dot(Wi, concat) + bi)
    ctilda = np.tanh(np.dot(Wc, concat) + bc)
    c_next = ft*c_prev + it* ctilda
    ot = sigmoid(np.dot(Wo, concat) + bo)
    a_next = ot*np.tanh(c_next)
    
    yt_pred = softmax(np.dot(Wy, a_next) + by)
    
    cache = (a_next, c_next, a_prev, c_prev, ft, it, ctilda, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

In [11]:
def lstm_forward(x, a0, parameters):
    
    caches = []
    
    n_x, m, Tx = x.shape
    n_y, n_a = parameters['Wy'].shape
    
    a = np.zeros((n_a, m, Tx))
    c = np.zeros((n_a, m, Tx))
    y = np.zeros((n_y, m, Tx))
    
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    for t in range(Tx):
        a_next, c_next, yt, cache = lstm_cell_forward(x[:,:,t], a_next, c_next, parameters)
        a[:,:,t] = a_next
        y[:,:,t] = yt
        c[:,:,t] = c_next
        caches.append(cache)
        
    caches = (caches, x)
    
    return a,y, c, caches